### Цифры в матричном виде

In [ ]:
numbers =['''
0 1 1 1 0
1 0 0 0 1
1 0 0 0 1
1 0 0 0 1
1 0 0 0 1
1 0 0 0 1
0 1 1 1 0
''','''
0 0 1 0 0
0 1 1 0 0
0 0 1 0 0
0 0 1 0 0
0 0 1 0 0
0 0 1 0 0
0 1 1 1 0
''','''
0 1 1 1 0
1 0 0 0 1
0 0 0 0 1
0 0 0 1 0
0 1 1 0 0
1 0 0 0 0
1 1 1 1 1
''','''
0 1 1 1 0
1 0 0 0 1
0 0 0 0 1
0 1 1 1 0
0 0 0 0 1
1 0 0 0 1
0 1 1 1 0
''','''
0 0 1 1 0
0 1 0 1 0
1 0 0 1 0
1 0 0 1 0
1 1 1 1 1
0 0 0 1 0
0 0 0 1 0
''','''
1 1 1 1 1
1 0 0 0 0
1 1 1 1 0
0 0 0 0 1
0 0 0 0 1
1 0 0 0 1
0 1 1 1 0
''','''
0 1 1 1 0
1 0 0 0 1
1 0 0 0 0
1 1 1 1 0
1 0 0 0 1
1 0 0 0 1
0 1 1 1 0
''','''
0 1 1 1 0
1 0 0 0 1
1 0 0 0 0
1 1 1 1 0
1 0 0 0 1
1 0 0 0 1
0 1 1 1 0
''','''
0 1 1 1 0
1 0 0 0 1
1 0 0 0 1
0 1 1 1 0
1 0 0 0 1
1 0 0 0 1
0 1 1 1 0
''','''
0 1 1 1 0
1 0 0 0 1
1 0 0 0 1
0 1 1 1 1
0 0 0 0 1
1 0 0 0 1
0 1 1 1 0'''
]

### Основная часть программы

In [ ]:
import os
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import argparse

def load_from_numbers():
  x,y = [], []
  for i, number in enumerate(numbers):
    number = number.replace('\n',' ')
    x.append([int(i) for i in number.split()])
    y.append(i)
  return x,y

def load_file(file, path = './'):
    with open(os.path.join(path,file),'r') as f:
        temp = f.read().replace('\n', ' ')
    return temp

def load_data(path):
    xdata = []
    ydata = []
    files = os.listdir(path)
    for file_name in files:
            temp = load_file(file_name,path)
            xdata.append([int(i) for i in temp.split()])
            ydata.append(file_name)
    return xdata, ydata

def multiply(x,y, num=1000): 
  #  "Увеличивает размер датасета"
    final_x = []
    final_y = []
    for temp_x,temp_y in zip(x,y):
        for i in range(num):
            final_x.append(temp_x)
            final_y.append(temp_y)
    return np.array(final_x),np.array(final_y)

def predict(model,number):
  #"Выполняет предсказание"
    return model.predict(np.asarray([number]))

def print_matrix(temp):
  #"Вывод пример"
    print(np.array(temp).reshape(7,5))

def train():
  #"Функция тренировки модели"
    x_data ,y_data = load_from_numbers()# Загружаем данные
    x,y = multiply(x_data,y_data)# Увеличиваем размер датасета

    y = np.array(y).reshape((-1,1))

    encoder = OneHotEncoder(categories='auto')
    labels_transformed = encoder.fit_transform(y).toarray()#Кодируем данные в OneHot вектор


    X_train, X_test, y_train, y_test = train_test_split(x, labels_transformed)#Разделяем данные на тренировочный и обучающий наборы
    #"Описываем модель"
    input_layer = keras.layers.Input(shape=(35,))
    d_1 = keras.layers.Dense(32, activation='relu')(input_layer)
    d_2 = keras.layers.Dense(16, activation='relu')(d_1)
    d_2 = keras.layers.Dense(y_train.shape[1], activation='softmax')(d_2)

    model = keras.models.Model(input_layer,d_2)
    model.summary()
#"Выбираем оптимизатор и метрики"
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
#"Подаем данные для обучения"
    model.fit(X_train, y_train, epochs=10, batch_size=128, validation_data = (X_test, y_test))
    for dat in x_data:
        print_matrix(dat)
        predicted_outputs = predict(model,dat)
        print('predicted number is',np.argmax(predicted_outputs, axis=1))
    model.save('model.h5')

def test():
    
    model = keras.models.load_model('model.h5')#Загружаем модель
    number =  [int(i) for i in test_file.split()]
    predicted_outputs = predict(model,number)#получаем предсказание
    print_matrix(number)
    print('predicted number is',np.argmax(predicted_outputs, axis=1))
    

funcs = {
    "train" : train,
    "test" : test
}

def main():
    parser = argparse.ArgumentParser(description='Modes')
    parser.add_argument('-m', action="store", dest="mode")
    args = parser.parse_args()
    funcs[args.mode]()


### Тренировка нейронной сети

In [ ]:
train()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 35)]              0         
                                                                 
 dense_15 (Dense)            (None, 32)                1152      
                                                                 
 dense_16 (Dense)            (None, 16)                528       
                                                                 
 dense_17 (Dense)            (None, 10)                170       
                                                                 
Total params: 1,850
Trainable params: 1,850
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
59/59 [==============================] - 1s 4ms/step - loss: 2.0846 - accuracy: 0.2991 - val_loss: 1.8084 - val_accuracy: 0.4836
Epoch 2/10
59/59 [==============================]

### Проверка нейронной сети

In [23]:
test_file = '''
0 1 1 1 0
1 0 0 0 1
0 0 0 0 1
0 1 1 1 0
0 0 0 0 1
1 0 0 0 1
0 1 1 1 0'''
test()

[[0 1 1 1 0]
 [1 0 0 0 1]
 [0 0 0 0 1]
 [0 1 1 1 0]
 [0 0 0 0 1]
 [1 0 0 0 1]
 [0 1 1 1 0]]
predicted number is [3]
